In [8]:
import os
import cv2
import numpy as np
import random
import shutil

# ---------------- Configuration ----------------
src_dir   = "dataset_clean2"     # Thư mục chứa ảnh gốc
dst_dir   = "augmented_dataset"  # Thư mục lưu ảnh sau augment
labels    = ["forward", "left", "right"]

# 1) Kiểm tra thư mục gốc
if not os.path.isdir(src_dir):
    raise FileNotFoundError(f"Không tìm thấy thư mục nguồn: {src_dir}")

# 2) Đếm số ảnh mỗi lớp, xác định target = max count
counts = {lbl: len([f for f in os.listdir(src_dir) if f.startswith(lbl + "_") and f.endswith(".jpg")])
          for lbl in labels}
max_count = max(counts.values())
print("Số ảnh ban đầu:", counts)
print("Target mỗi lớp sau augment =", max_count)

# 3) Tạo lại thư mục đích
if os.path.exists(dst_dir):
    shutil.rmtree(dst_dir)
os.makedirs(dst_dir, exist_ok=True)

# 4) Hàm augment – chỉ Brightness (không shift, không rotate)
def augment_brightness(img):
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    delta = random.randint(-40, 40)
    hsv[:, :, 2] = np.clip(hsv[:, :, 2] + delta, 0, 255)
    return cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

# 5) Chuẩn bị bộ đếm ID cho mỗi lớp
next_id = {lbl: 0 for lbl in labels}

# 6) Copy ảnh gốc & augment
for lbl in labels:
    files = [f for f in os.listdir(src_dir) if f.startswith(lbl + "_") and f.endswith(".jpg")]
    print(f"\nXử lý lớp '{lbl}': có {len(files)} ảnh gốc")

    # 6.1 Copy ảnh gốc
    for f in files:
        img = cv2.imread(os.path.join(src_dir, f))
        out_name = f"{lbl}_{next_id[lbl]}.jpg"
        cv2.imwrite(os.path.join(dst_dir, out_name), img)
        next_id[lbl] += 1

    # 6.2 Augment thêm cho đủ max_count
    while next_id[lbl] < max_count:
        src_file = random.choice(files)
        img = cv2.imread(os.path.join(src_dir, src_file))

        if random.random() < 0.3:
            # (A) Flip ngang + đổi nhãn left↔right
            img_flip = cv2.flip(img, 1)
            new_lbl = lbl
            if lbl == "left":
                new_lbl = "right"
            elif lbl == "right":
                new_lbl = "left"
            out_name = f"{new_lbl}_{next_id[new_lbl]}.jpg"
            cv2.imwrite(os.path.join(dst_dir, out_name), img_flip)
            next_id[new_lbl] += 1
        else:
            # (B) Brightness augment
            img_bright = augment_brightness(img)
            out_name = f"{lbl}_{next_id[lbl]}.jpg"
            cv2.imwrite(os.path.join(dst_dir, out_name), img_bright)
            next_id[lbl] += 1

# 7) In ra kết quả cuối
final_counts = {lbl: len([f for f in os.listdir(dst_dir) if f.startswith(lbl + "_")])
                for lbl in labels}
print("\nSố ảnh sau augment lên bằng nhau:", final_counts)


Số ảnh ban đầu: {'forward': 723, 'left': 58, 'right': 55}
Target mỗi lớp sau augment = 723

Xử lý lớp 'forward': có 723 ảnh gốc

Xử lý lớp 'left': có 58 ảnh gốc

Xử lý lớp 'right': có 55 ảnh gốc

Số ảnh sau augment lên bằng nhau: {'forward': 723, 'left': 894, 'right': 723}
